In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk

In [8]:


# Read in the data
df1 = pd.read_csv('data/Reddit_Data.csv',encoding_errors='ignore')
df2 = pd.read_csv('data/test.csv',encoding_errors='ignore')
df3 = pd.read_csv('data/train.csv',encoding_errors='ignore')
df4 = pd.read_csv('data/Twitter_Data.csv',encoding_errors='ignore')
# df5 = pd.read_csv('data/train.tsv',encoding_errors='ignore', delimiter='\t')
# df6 = pd.read_csv('data/training.1600000.processed.noemoticon.csv',encoding_errors='ignore')

# print out value counts for sentiment or category column of each dataframe
print("df1\n",df1["category"].value_counts())
print("df2\n",df2["sentiment"].value_counts())
print("df3\n",df3["sentiment"].value_counts())
print("df4\n",df4["category"].value_counts())
# print("df4\n",df5["Sentiment"].value_counts())
# print("df6\n",df6["polarity of tweet"].value_counts())


df6

df1
  1    15830
 0    13142
-1     8277
Name: category, dtype: int64
df2
 neutral     1430
positive    1103
negative    1001
Name: sentiment, dtype: int64
df3
 neutral     11118
positive     8582
negative     7781
Name: sentiment, dtype: int64
df4
  1.0    72250
 0.0    55213
-1.0    35510
Name: category, dtype: int64
df6
 0    799996
4    248576
Name: polarity of tweet, dtype: int64


,polarity of tweet,id of the tweet,date of the tweet,query,user,text of the tweet
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
...,...,...,...,...,...,...
1048567,4,1960186342,Fri May 29 07:33:44 PDT 2009,NO_QUERY,Madelinedugganx,My GrandMa is making Dinenr with my Mum
1048568,4,1960186409,Fri May 29 07:33:43 PDT 2009,NO_QUERY,OffRoad_Dude,Mid-morning snack time... A bowl of cheese noo...
1048569,4,1960186429,Fri May 29 07:33:44 PDT 2009,NO_QUERY,Falchion,@ShaDeLa same here say it like from the Termi...
1048570,4,1960186445,Fri May 29 07:33:44 PDT 2009,NO_QUERY,jonasobsessedx,@DestinyHope92 im great thaanks wbuu?


In [3]:
df1_std = df1.copy()
df2_std = df2.copy()
df3_std = df3.copy()
df4_std = df4.copy()
# df5_std = df5.copy()
df6_std = df6.copy()

df1_std.columns = ["text","label"]
df1_std.dropna(axis='index',inplace=True)

df2_std = df2_std.loc[:,["text","sentiment"]]
df2_std.dropna(axis='index',inplace=True)
df2_std.columns = ["text","label"]
df2_std["label"] = df2_std["label"].map({"positive":'1',"neutral":'0',"negative":'-1'})

df3_std = df3_std.loc[:,["text","sentiment"]]
df3_std.dropna(axis='index',inplace=True)
df3_std.columns = ["text","label"]
df3_std["label"] = df3_std["label"].map({"positive":'1',"neutral":'0',"negative":'-1'})
df3_std

df4_std = df4_std.loc[:,["clean_text","category"]]
df4_std.dropna(axis='index',inplace=True)
df4_std.columns = ["text","label"]
df4_std["label"] = df4_std["label"].map({1.0:'1',0.0:'0',-1.0:'-1'})
df4_std

# df5_std = df5_std.loc[:,["text","Sentiment"]]
# df5_std.columns = ["text","label"]
# df5_std["label"] = df5_std["label"].map({"positive":1,"neutral":0,"negative":-1})
# df5_std

# df6_std = df6_std.loc[:,["text of the tweet","polarity of tweet"]]
# df6_std.dropna(axis='index',inplace=True)
# df6_std.columns = ["text","label"]
# df6_std["label"] = df6_std["label"].map({4:'1',1:'-1'})
# df6_std

# combine df1_std and df2_std dataframes
combined_df = pd.concat([df1_std,df2_std,df3_std,df4_std],axis=0)
combined_df

# print out the shapes of all dataframes
print("df1_std",df1_std.shape)
print("df2_std",df2_std.shape)
print("df3_std",df3_std.shape)
print("df4_std",df4_std.shape)
# print("df5_std",df5_std.shape)
print("combined_df",combined_df.shape)

combined_df



df1_std (37149, 2)
df2_std (3534, 2)
df3_std (27480, 2)
df4_std (162969, 2)
combined_df (231132, 2)


,text,label
0,family mormon have never tried explain them t...,1
1,buddhism has very much lot compatible with chr...,1
2,seriously don say thing first all they won get...,-1
3,what you have learned yours and only yours wha...,0
4,for your own benefit you may want read living ...,1
...,...,...
162975,why these 456 crores paid neerav modi not reco...,-1
162976,dear rss terrorist payal gawar what about modi...,-1
162977,did you cover her interaction forum where she ...,0
162978,there big project came into india modi dream p...,0


In [4]:
import multiprocessing
from gensim.models import Word2Vec



# make all the text lowercase
combined_df.text = combined_df.text.str.lower()

# save the data as a list of tokenized sentences
data = combined_df.text.apply(nltk.word_tokenize)
data_lines = data.tolist()

## stuff if we wanna do NGRAM related stuff
# NGRAM = 3
# spooky_lines = data.apply(lambda x: ["<s>"]*(NGRAM-1) + x + ["</s>"]*(NGRAM-1)).tolist()

# with open(f"spooky_lines_{NGRAM}.txt","w",encoding="utf-8") as f:
#     for line in combined_df["text"]:
#         f.write("<line> "*(NGRAM-1) + line + " </line>"*(NGRAM-1)+"\n")

# The dimension of word embedding. 
# This variable will be used throughout the program
# you may vary this as you desire
# EMBEDDINGS_SIZE = 100
for EMBEDDINGS_SIZE in (25,50,100,200):

    # Train the Word2Vec model from Gensim. 
    sg = 1
    window = 5
    vector_size = EMBEDDINGS_SIZE
    min_count = 1

    embeddings = Word2Vec(sentences=data_lines, vector_size=vector_size, window=window, min_count=min_count, sg=sg, workers=multiprocessing.cpu_count())
    # save file in txt format, then load later if you wish.
    embeddings.wv.save_word2vec_format(f'embeddings_{EMBEDDINGS_SIZE}.txt', binary=False)


In [5]:
# if you save your Word2Vec as the variable model, this will 
# print out the vocabulary size
print('Vocab size: {}'.format(len(embeddings.wv)))

Vocab size: 151279
